In [23]:
import os
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

import import_ipynb
from data_preprocessing import preprocess_all_files

# Split the data into training and testing sets at the file level
test_percentage = 0.2
all_data = preprocess_all_files()

num_test_files = int(len(all_data) * test_percentage)
test_data = all_data[:num_test_files]
train_data = all_data[num_test_files:]

print(all_data[0])
# print("Test Data Type: %s" % type(all_data[0]))
# print("test Data Count: %s" % len(train_data))
# print(test_data)

# LSTM Model Training
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(train_data[0][0].shape[1], 1)))
model.add(Dense(units=train_data[0][1].shape[1], activation='linear'))


model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

for X_train, y_train in train_data:
    # Expand dimensions for LSTM input
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32)

# Now you can loop through 'test_data' to evaluate your model on the test set
for X_test, y_test in test_data:
    # Expand dimensions for LSTM input
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    # Evaluate the model
    loss, mae = model.evaluate(X_test, y_test)
    print(f"Mean Absolute Error on Test Set: {mae}")

(array([[ 1.        ,  0.22533894, -0.37617263, ..., -0.03667128,
        -0.06526835,  0.02950843],
       [ 1.        ,  0.22355048, -0.37506536, ..., -0.02676817,
        -0.06219667,  0.03502281],
       [ 1.        ,  0.22865094, -0.3745335 , ..., -0.01353437,
        -0.04698311,  0.03366825],
       ...,
       [ 4.        ,  0.92228913, -0.14020322, ..., -0.20870168,
         0.34647736, -0.30882737],
       [ 4.        ,  0.9251055 , -0.16115499, ..., -0.23629583,
         0.20737056, -0.29414824],
       [ 4.        ,  0.90814066, -0.18164802, ..., -0.32424653,
        -0.35834023, -0.2307228 ]]), {'CGUID': '1018', 'Date': '20230916_01:16:54:377', 'ShotResult': [7.697021961212158, 25.65673828125, -0.608303964138031, 8.458624733975739e-07, -2.7872023582458496, 2.058823585510254, 97.85504913330078, 0.3499999940395355, 0.17000000178813934, 1.0]})


AttributeError: 'dict' object has no attribute 'shape'